In [ ]:
import json

klids_m_gt = []
klids=[]

with open('inn_data/klidsFraGT.json', 'r') as f:
    klids_m_gt_json = json.load(f)
    for element in klids_m_gt_json:
        klids_m_gt.append(element['KartleggingsenhetId'])

with open('inn_data/alle_m005_ids.csv', 'r') as f:
    klids_tmp = f.read().splitlines()[1:]
    for v in klids_tmp:
        klids.append(int(v))
klids
#klids_m_gt
klids_no_gt = list(set(klids) - set(klids_m_gt))
klids_no_gt
#print(klids_no_gt.sort())

# (#185: Sjekker langkoder på Hovedtyper)

In [ ]:
 # Fetching hovedtype-kortkode and langkode from database
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("C:/Projects/nin3-kode-api/NiN3.WebApi/nin3kodeapi.db")
query = "SELECT Langkode, Kode as HTKode FROM Hovedtype"  # Write your SQLite query
df_db = pd.read_sql_query(query, conn)  # Execute the query and store the result in a DataFrame
conn.close()  # Close the database connection

# Create a new column langkode_parent
df_db['langkode_parent'] = df_db['Langkode'].str.split('-', n=6).str[:6].str.join('-')

#print(df_db)  # Print the DataFrame


## Fetching hovedtype-kortkode(splitted on parent) and langkode from excel 
plk_htk = nin3_typer[['Langkode', 'HTKode']] 
plk_htk = plk_htk.astype(str) # setting all columns to string
plk_htk = plk_htk.applymap(lambda x: x.strip() if isinstance(x, str) else x) # removing whitespace
plk_htk['langkode_parent_xl'] = plk_htk['Langkode'].str.split('-', n=6).str[:6].str.join('-')
plk_htk['HTKode'] = plk_htk['HTKode'].str.replace('_', '-')
plk_htk = plk_htk.drop('Langkode', axis=1)
plk_htk = plk_htk.drop_duplicates(subset=['HTKode', 'langkode_parent_xl'])

ht_kode_parentCheck = df_db.merge(plk_htk, on='HTKode', how='left') # dette resultatet gir en del miss fra Excel-siden
ht_kode_parentCheck # (Langkode     HTKode	    langkode_parent	    langkode_parent_xl)
ht_kode_parentCheck.to_csv('kladd_debugging/htk_langkode_parent_matchCheck.csv', sep=';', index=False)


#plk_htk = 

In [1]:
csv = """grunntyper.csv
grunntype_variabeltrinn_mapping.csv
hovedtype.csv
hovedtypegrupper.csv
hovedtypegruppe_hovedtype_mapping.csv
hovedtype_grunntype_mapping.csv
hovedtype_variabeltrinn_mapping.csv
htg_ht_gt_mapping.csv
konvertering_gt_v30.csv
konvertering_htg_v30.csv
konvertering_ht_v30.csv
M005.csv
m005_grunntype_mapping.csv
m005_hovedtype_mapping.csv
m020.csv
m020_grunntype_mapping.csv
m020_hovedtype_mapping.csv
m050.csv
m050_grunntype_mapping.csv
m050_hovedtype_mapping.csv
maaleskala_enhet.csv
maaleskala_trinn.csv
type.csv
typeklasser_langkode_mapping.csv
type_htg_mapping.csv
variabel.csv
variabelnavn.csv
variabelnavn_maaleskala_mapping.csv
variabelnavn_variabel_mapping.csv"""
lines = csv.split("\n")
markdown = ""
for line in lines:
    markdown += f"- [ ] {line}\n"

print(markdown)


- [ ] grunntyper.csv
- [ ] grunntype_variabeltrinn_mapping.csv
- [ ] hovedtype.csv
- [ ] hovedtypegrupper.csv
- [ ] hovedtypegruppe_hovedtype_mapping.csv
- [ ] hovedtype_grunntype_mapping.csv
- [ ] hovedtype_variabeltrinn_mapping.csv
- [ ] htg_ht_gt_mapping.csv
- [ ] konvertering_gt_v30.csv
- [ ] konvertering_htg_v30.csv
- [ ] konvertering_ht_v30.csv
- [ ] M005.csv
- [ ] m005_grunntype_mapping.csv
- [ ] m005_hovedtype_mapping.csv
- [ ] m020.csv
- [ ] m020_grunntype_mapping.csv
- [ ] m020_hovedtype_mapping.csv
- [ ] m050.csv
- [ ] m050_grunntype_mapping.csv
- [ ] m050_hovedtype_mapping.csv
- [ ] maaleskala_enhet.csv
- [ ] maaleskala_trinn.csv
- [ ] type.csv
- [ ] typeklasser_langkode_mapping.csv
- [ ] type_htg_mapping.csv
- [ ] variabel.csv
- [ ] variabelnavn.csv
- [ ] variabelnavn_maaleskala_mapping.csv
- [ ] variabelnavn_variabel_mapping.csv



## (#197) Sjekker for manglende GT: Alle unike Grunntype navn from excel

In [2]:
# 
file_path = '../tmp/GTnavn.txt'

# Read the file and store each line as an element in a list
with open(file_path, 'r') as file:
    lines = file.read().splitlines()

# Count the number of unique elements in the list
unique_elements = len(set(lines))

unique_elements


1395

## Henter alle navn fra sqlite databasen

In [39]:
import pandas as pd
import sqlite3
file_path = '../tmp/GTnavn.txt'

# Read the file and store each line as an element in a list
with open(file_path, 'r') as file:
    lines = file.read().splitlines()

# Count the number of unique elements in the list
unique_elements = len(set(lines))

print("Antall unike grunntypenavn fra excelark: "+str(unique_elements))

conn = sqlite3.connect("C:/Projects/nin3-kode-api/NiN3.WebApi/nin3kodeapi.db")
#conn = sqlite3.connect("C:/temp/nin3kodeapi.db")
query = "SELECT lower(navn) as navn FROM Grunntype"  # Write your SQLite query
df_gtnavn = pd.read_sql_query(query, conn)  # Execute the query and store the result in a DataFrame
#df_gtnavn = df_gtnavn.astype(str) 
  # Close the database connection
#print("Antall navn i basen:"+df_gtnavn.size)
#print("Antall navn i basen: " + df_gtnavn.shape[0])
query = "select count(distinct(Navn)) as AntUnikeNavn, count(navn) as AntNavn from Grunntype"
df_gtnavn_count = pd.read_sql_query(query, conn)  # Execute the query and store the result in a DataFrame
print("\nAntall unike grunntype navn i basen:")
display(df_gtnavn_count)
query 
#print("Antall unike navn i basen:"+df_gtnavn['navn'].nunique())
duplicate_names = df_gtnavn[df_gtnavn.duplicated('navn')]['navn']
print("Dupilkate grunntype navn i basen:")
display(duplicate_names)
duplicate_names_str = ', '.join(["'" + name + "'" for name in duplicate_names])
#display(duplicate_names_str)
query = f"select Kode, Navn from Grunntype where lower(Navn) in ({duplicate_names_str})"
df_koderDuplikat = pd.read_sql_query(query, conn)  # Execute the query and store the result in a DataFrame
df_koderDuplikat.to_html("../tmp/gt_duplikat.html", index=False)
print("*** Koder til duplikatene:")
display(df_koderDuplikat)
df_gtnavn.to_csv('../tmp/GTnavnFraDB.csv', index=False)
df_gtnavn.shape
conn.close()

Antall unike grunntypenavn fra excelark: 1395

Antall unike grunntype navn i basen:


,AntUnikeNavn,AntNavn
0,1395,1401


Dupilkate grunntype navn i basen:


133     nedskåret hei- og fjellskog-dallandskap med by...
135             nedskåret fjellhei-dallandskap med innsjø
250     middels eksponert og middels kupert kystslette...
251     middels eksponert og middels kupert våtmarkspr...
1094    temmelig til ekstremt kalkrikt nedre tuenivå i...
1335                                            vedrester
Name: navn, dtype: object

*** Koder til duplikatene:


,Kode,Navn
0,I-0-02-071,Nedskåret hei- og fjellskog-dallandskap med by...
1,I-0-02-080,Nedskåret hei- og fjellskog-dallandskap med by...
2,I-0-02-081,Nedskåret fjellhei-dallandskap med innsjø
3,I-0-02-082,Nedskåret fjellhei-dallandskap med innsjø
4,K-0-03-025,Middels eksponert og middels kupert kystslette...
5,K-0-03-026,Middels eksponert og middels kupert våtmarkspr...
6,K-0-03-031,Middels eksponert og middels kupert kystslette...
7,K-0-03-032,Middels eksponert og middels kupert våtmarkspr...
8,V-A-01-20,Temmelig til ekstremt kalkrikt nedre tuenivå i...
9,V-A-01-29,Temmelig til ekstremt kalkrikt nedre tuenivå i...
